![logo](https://github.com/koulali/ceg1705/blob/main/ceg1705_logo_notebook.png?raw=true)


# Practical 2: GNSS in the field

## Sub-group information :

Before starting your analysis, please input your group's name in the following cell:

👇 ✍🏻

In [ ]:
# Your group name [A-G]
subgroup_name = "..."

N.B 

- Cells with "👇 ✍🏻" indicate that you have to input your answer in the following cell.
- Cells with "✏️ Your answer here" where you must write your comments.
- Make sure you compile all cells following the notebook order.


Now run the following cell 👇 to install the required libraries.

In [ ]:
!pip install folium
!pip install openpyxl
!pip install rich


❗Now click on `Kernel` in the Menu bar above, then click on `Restart` to restart the kernel. Then start your analysis. 

In [1]:
from numpy import sign,radians,sin,cos,sqrt,power,average,std
import pandas as pd
import folium
from rich.table import Table
from rich.console import Console

After returning from the field, you have now to start analysing and reporting your groups results. This notebook will help you to go step by step through the analysis of your data. 


### (a) Coordinate entry and comparison

The sheet below allows you to input your data, enter **your observed coordinates** in the relevant cells (if necessary for your receiver, subtract the antenna height from your ellipsoidal heights before entering the marker height). Remember that longitudes west of Greenwich (i.e. the prime meridian) should be negative (in the deg column only; the notebook has been set up to take care of the rest). If your receiver provided decimals of a minute, leave the sec column with 0 value.

In [2]:
# load true coordinates
ref = pd.read_csv('https://raw.githubusercontent.com/koulali/ceg1705/main/town_moor_benchmarks.txt',delim_whitespace=True,names=['lat','lon','h'])
ref

,lat,lon,h
0,54.983996,-1.617997,113.920
1,54.984250,-1.618031,114.300
2,54.984491,-1.618055,114.211
3,54.984727,-1.618085,113.987
4,54.984882,-1.618107,113.621
5,54.985134,-1.618379,113.739
6,54.985211,-1.618699,114.412
7,54.985280,-1.618982,114.974
8,54.985342,-1.619238,115.448
9,54.985410,-1.619515,115.946


In [5]:
# load your data (excel file)
data = pd.read_excel('data.xlsx', index_col=0)
data

,Lat.deg,Lat.min,Lat.sec,Lon.deg,Lon.min,Lon.sec,H,CQ[plan],CQ[height],CQ[3D]
TM4,54,59,2.4430,-1,37,4.698,117.30,5.9,7.1,0
TM5,54,59,3.3170,-1,37,4.826,116.85,5.1,6.9,1
TM6,54,59,4.2190,-1,37,4.971,117.40,5.2,7.9,2
TM7,54,59,4.9790,-1,37,5.634,118.30,5.7,8.4,3
TM8,54,59,5.6830,-1,37,5.175,118.74,5.7,7.1,4
TM9,54,59,6.5100,-1,37,6.110,118.32,3.7,6.8,5
TM10,54,59,6.6835,-1,37,7.307,115.37,5.0,6.5,6
TM11,54,59,6.9940,-1,37,8.254,117.86,2.4,4.0,7
TM12,54,59,7.3220,-1,37,9.288,115.49,5.5,6.9,8
TM13,54,59,7.5300,-1,37,10.418,114.98,5.6,9.4,9


Let's convert you coordinates to decimal degree. We prepared a function to do this.

In [6]:
# Latitude
s = sign(data['Lat.deg'])
lat_dec = data['Lat.deg'] + s*data['Lat.min']/60 + s*data['Lat.sec']/3600

# Longitude
s = sign(data['Lon.deg'])
lon_dec = data['Lon.deg'] + s*data['Lon.min']/60 + s*data['Lon.sec']/3600

# Height
h = data['H']

Now Let's plot your coordinates to check on map 😜😜

In [9]:
# Create interactive map with default basemap
map_moor = folium.Map(location=[54.98488, -1.618106],zoom_start=17)
for i in range(len(lat_dec)):
    folium.Marker( location=[ lat_dec.iloc[i], lon_dec.iloc[i] ],popup=data.index[i], fill_color='#43d9de', radius=8 ).add_to( map_moor )

map_moor

The precise surveyed coordinates are given in the variables `ref['lat']`, `ref['lon']` and `ref['h']`. You must compute the difference (observed minus true) **in metres**.


Let's first compute the difference in degrees and store in the variables `diff_lat` and `diff_lon`.

To do an array difference you simply subtract; For example for computing latitude differences:        
```
    diff_lat = lat_dec.values - ref['lat'].values
```

👇 ✍🏻

In [ ]:
diff_lat = lat_dec.values - ref['lat'].values
diff_lon = lon_dec.values - ref['lon'].values
diff_h_meters = h.values - ref['h'].values

The **difference in metres** is the **difference in radians** times the radius of curvature in the meridian ($\nu$ [nu]).


To convert from degrees to radians, we use the function `radians` from the module numpy.

e.g 

    lat_radians = np.radians(lat_degrees)


👇 ✍🏻

In [ ]:
diff_lat_rad = radians(diff_lat)

I calculated for you the radius of curvature $\nu$ (`nu`), all you need is to run the following cell.

In [ ]:
a = 6378137.000
b = 6356752.314

e_2 = (a**2-b**2)/a**2
phi_A = average(radians(ref['lat']))
nu = a/sqrt(1-e_2*sin(phi_A)**2)

Now, we're ready to calculate the latitude differences in meters. We store then in the variables `diff_lat_meters`.


👇 ✍🏻

In [ ]:
diff_lat_meters = diff_lat_rad*nu

For the longitude difference, make use of the longitude in radians. First, we convert the differences from degress to radians:


👇 ✍🏻

In [ ]:
diff_lon_rad = radians(diff_lon)

The difference in metres is the difference in radians, times the radius of curvature in the prime vertical ($\rho$ or `rho`), times the cosine of the latitude $\phi_A$.  The combined quantity $\rho cos(\phi_A)$ is computed for you in the cell below.  

In [ ]:
rho_cos = a*(1-e_2)/power(1-e_2*sin(phi_A)**2,1.5)*cos(phi_A)

The longitude differences in meters:


👇 ✍🏻

In [ ]:
diff_lon_meters = diff_lon_rad*rho_cos

In [ ]:
table = Table(title="Differences Lat. Lon. in meters")
table.add_column("Diff. Lat.[m]")
table.add_column("Diff. Lon.[m]")
table.add_column("Diff. height[m]")
for i in range(10):
    table.add_row(str(diff_lat_meters[i]),str(diff_lon_meters[i]),str(diff_h_meters[i]))
console = Console()
console.print(table)

You should now have a set of numbers giving the distances between your observed coordinates and the true ones.  Expect these differences to be anywhere between a few millimetres and a few tens of metres in size.  If your receiver could not observe a particular coordinate, do not compute a coordinate difference (leave the relevant cells blank).  If there are any differences which are much larger than the others, check your data and if necessary discard these outliers.

### (a) Statistics of coordinate differences

Compute the mean coordinate differences $\mu$ in latitude, longitude and height. For this you need the function numpy `average()` which computes the arithmetic mean. These values represent the accuracy (bias or systematic error) of your receiver today, i.e. the average offset between your results and the truth.

e.g of using the average function:

    avg_diff_lat = average(diff_lat_meters)
    


👇 ✍🏻

In [ ]:
avg_diff_lat = average(diff_lat_meters)
avg_diff_lon = average(diff_lon_meters)
avg_diff_h = average(diff_h_meters)

Compute the standard deviations $\sigma$ of the coordinate differences in latitude, longitude and height.  For this you need the function `std()` which computes the standard deviation of the values in the specified array, for example:

        std_diff_lat = std(diff_lat_meters)   
    
    
These values represent the precision (random error) of your receiver today, i.e. the scatter of your results about their systematic error (mean).


👇 ✍🏻

In [ ]:
std_diff_lat = std(diff_lat_meters)
std_diff_lon = std(diff_lon_meters)
std_diff_h = std(diff_h_meters)

Compute the root mean squares (RMS) of the coordinate differences in latitude, longitude and height, given by the formula : $\sqrt{\mu^2 + \sigma^2}$ . This is an overall representation of the coordinate quality (accuracy and precision combined). Use the function `sqrt` to calculate the square-root. 


👇 ✍🏻

In [ ]:
rms_diff_lat = sqrt(avg_diff_lat**2 + std_diff_lat**2)
rms_diff_lon = sqrt(avg_diff_lon**2 + std_diff_lon**2)
rms_diff_h = sqrt(avg_diff_h**2 + std_diff_h**2)

In [ ]:
table = Table(title="Statistics of differences")
table.add_column("Coord.")
table.add_column("avg(m)")
table.add_column("std(m)")
table.add_column("rms(m)")
table.add_row("Latitude",str(avg_diff_lat),str(std_diff_lat),str(rms_diff_lat))
table.add_row("Longitude",str(avg_diff_lon),str(std_diff_lon),str(rms_diff_lon))
table.add_row("Height",str(avg_diff_h),str(std_diff_h),str(rms_diff_h))
console = Console()
console.print(table)

You're done. Time to submit your work 👉.

Share your notebook, then submit the link via canvas (similar to Practical 1). You will present these results at Seminar 1.  
If a sub-group has more than one member, all should be present, although you may delegate the act of speaking to one person if you prefer.  
You should be prepared to speak informally for 1-2 minutes and should include a brief overview of the features and ease of use of your receiver 
from your experience in the field, and a commentary on the coordinate differences and their statistics.